In [14]:
import ROOT
import numpy as np

In [15]:
def linspace(start, end, steps):
    values = []
    for i in range(steps):
        values.append(start + i*(end-start)/(steps-1))
    return values

In [ ]:
mu_collisions_per_BC = 200               #The parameter mu that is the expectation value of collisions per BC, which can be altered manually
particles_per_collision = 75             #The average number of particles that are created in each collision. Between 50 and 100
ratio_of_particles_through_HGTD = 0.01    #My guess of the ratio of particles created in a BC that cross through HGTD. Should probably be between 0.1 and 1
Noisereadings_per_BC = 3.6*10**6

BCs_to_simulate = 100                      #How many subsequent bunch crossings the code simulates

real_signal_mu = 100                     #The real value of the parameter mu in the signal enery curve for detected particles
real_signal_sigma = 25                   #Since at the start

#model_signal_mu_spread_interval_length = 50 * 2
model_signal_mu_step_count = 20

model_signal_mus = linspace(1, real_signal_mu, model_signal_mu_step_count)
#model_signal_sigma = linspace(0, real_signal_mu/4, model_signal_mu_step_count)  #Since sigma is supposed to be proportional to mu, and is 25 at the start when mu=100

treshold_lower = 0                        #We test different tresholds for the plot
treshold_upper = 60
treshold_step_count = 20
treshold_values =  linspace(treshold_lower, treshold_upper, treshold_step_count)

noise_mu = 0
noise_sigma = 5

print(model_signal_mus)
print(treshold_values)

model_noise = ROOT.TF1("ElecNoiseBg", "TMath::Gaus(x, %f, %f)" % (noise_mu, noise_sigma), -100, 500)
model_noise.SetNpx(10000)


"""
# more paramaters
simulation_time = 1 #seconds
bunch_crossings_per_second = 40 * 1e6
collisions_per_bunch_crossing = en fördelning
particles_per_collision = en fördelning
particles_reaching_the_sensor = en fördelning vi inte känner till och inte är vår uppgift att känna till
 
"""

prng = ROOT.TRandom3(123)

error_against_mu_treshold_plot = ROOT.TGraph2D()         #Function for plotting the deviation between the calculated integrated luminosity and the actual integrated luminosity, as a function of the parameter mu in the energy landau distribution and the treshold
error_against_mu_treshold_logplot = ROOT.TGraph2D()
point_index = 0

for model_signal_mu in model_signal_mus:
    model_signal = ROOT.TF1("Signal", "TMath::Landau(x, %f, %f)" % (model_signal_mu, model_signal_mu/4), -100, 500)
    model_signal.SetNpx(10000)
    for treshold_value in treshold_values:
        model_expected_signal_above = 1 - model_signal.Integral(-100, treshold_value)/model_signal.Integral(-100, 500)
        model_expected_noise_above = 1 - model_noise.Integral(-100, treshold_value)/model_noise.Integral(-100, 500)
        #print("Model expected signal above " + str(model_expected_signal_above))
        #print("Model expected noise above " + str(model_expected_noise_above))
        
        noise_count = BCs_to_simulate * Noisereadings_per_BC
        signal_count = BCs_to_simulate * mu_collisions_per_BC * particles_per_collision * ratio_of_particles_through_HGTD

        noise_above_count = 0
        signal_above_count = 0
        large_interval_noise_above_count = 0

        for i in range(int(signal_count)):
            signal = prng.Landau(real_signal_mu, real_signal_sigma)
            if signal > treshold_value:
                signal_above_count += 1

        for i in range(int(noise_count/1000)):               #Since noise_count is so large we only run over 1/1000 of it to calculate the amount of noise to goes over the treshold, then multiply by a 1000
            noise = prng.Gaus(noise_mu, noise_sigma)
            if noise > treshold_value:
                noise_above_count += 1
        noise_above_count = noise_above_count *1000
        
        for i in range(int(noise_count/1000)):               #Since noise_count is so large we only run over 1/1000 of it to calculate the amount of noise to goes over the treshold, then multiply by a 1000
            noise = prng.Gaus(noise_mu, noise_sigma)
            if noise > treshold_value:
                 large_interval_noise_above_count += 1
        large_interval_noise_above_count = large_interval_noise_above_count *1000
                
        total_above_count = noise_above_count + signal_above_count
        
        """
        Antag att vi vet:
            total_above_count
            model_expected_signal_above
            model_expected_noise_above
            
            large_interval_noise_above_count
            
        Hur ska vi räkna?
        Vi vill beräkna ett värde som är så nära signal_count som möjligt.
        """
        #noise_count_error = 0.0            #(Ohm sa typ att denna är 0) Hur stort är felet är skattningen av bruset från det stora intervallet 
        
        #expected_noise_count = Noisereadings_per_BC
        #expected_signal_count = mu_collisions_per_BC * particles_per_collision * ratio_of_particles_through_HGTD
        #expected_event_ratio = mu_collisions_per_BC * particles_per_collision * ratio_of_particles_through_HGTD/Noisereadings_per_BC
        
        #First method of computing signal count, assuming we know noise_above_count from larger interval
        computed_signal_count = (total_above_count - large_interval_noise_above_count)/model_expected_signal_above
        
        #Second method of computing signal count, which doesn't assume we know the amount of noise
        new_computed_signal_count = total_above_count/model_expected_signal_above
        
        # Compare with signal_count which is the correct value
        #print([model_signal_mu, treshold_value])
        #print("Computed signal above over signal count assuming we know event_ratio: " + str(computed_signal_count/signal_count))
        #print("Total above over signal count " + str(total_above_count/signal_count))
        #print("Computed signal above over signal count NOT assuming we know event_ratio: " + str(new_computed_signal_count/signal_count))
        
        #TFor the plot
        deviation = computed_signal_count/signal_count      #Calculate the deviation with both ways of computing the integrated luminosity
        alt_deviation = new_computed_signal_count/signal_count
        
        log_deviation = np.log(0.00000001 + abs(1 - computed_signal_count/signal_count))      #Also calculating the log of the deviation
        alt_log_deviation = np.log(0.00000001 + abs(1 - new_computed_signal_count/signal_count))
        
        error_against_mu_treshold_plot.SetPoint(point_index, model_signal_mu/noise_sigma, treshold_value, deviation)           #Adding data to both the logarithmic and non-logarithmic plot
        error_against_mu_treshold_logplot.SetPoint(point_index, -model_signal_mu/noise_sigma, -treshold_value, log_deviation)   #I made the mu and treshold values negative to "flip" the axes in the plot for better visual inspection
        
        point_index += 1
        
#Försök till plot (just nu kan man bara plotta antingen log plotten eller icke-log plotten)        
canvas = ROOT.TCanvas("I dont even know where this will show up", "not this either", 800, 600)
#error_against_mu_treshold_plot.Draw("TRI1")
error_against_mu_treshold_logplot.Draw("TRI1")
#error_against_mu_treshold_plot.SetTitle("Plot of deviation between calculated and actual integrated luminosity, using the large interval to estimate and subtract noise above treshold. S/B is varied by varying mu for the signal landau-distr. Real sigma mu = "+ str(real_signal_mu) + "; model sigma mu; treshold; deviation");
error_against_mu_treshold_logplot.SetTitle("Deviation between calculated and actual integrated luminosity, using large interval to subtract noise; S/B; treshold; logarithmic deviation");

canvas.Draw()


[1.0, 6.2105263157894735, 11.421052631578947, 16.63157894736842, 21.842105263157894, 27.05263157894737, 32.26315789473684, 37.473684210526315, 42.68421052631579, 47.89473684210526, 53.10526315789474, 58.31578947368421, 63.526315789473685, 68.73684210526316, 73.94736842105263, 79.15789473684211, 84.36842105263158, 89.57894736842105, 94.78947368421052, 100.0]
[0.0, 3.1578947368421053, 6.315789473684211, 9.473684210526315, 12.631578947368421, 15.789473684210526, 18.94736842105263, 22.105263157894736, 25.263157894736842, 28.42105263157895, 31.57894736842105, 34.73684210526316, 37.89473684210526, 41.05263157894737, 44.21052631578947, 47.36842105263158, 50.526315789473685, 53.68421052631579, 56.8421052631579, 60.0]
